# MLB Moneyline Regression Analysis

This notebook uses the mlbgame API to predict MLB moneylines from current standings and starting pitcher ERA.

**Mission:** Predict Moneylines from current standings and starting pitcher ERA

**Complexity:** Small

**Backtested:** No

**Chances of beating the books:** Low

In [7]:
import pybaseball as pyball
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette('husl')

In [8]:
from pybaseball import standings, batting_stats, pitching_stats, team_batting, team_pitching, team_game_logs, statcast
# see all columns
pd.set_option('display.max_columns', None)

statcast_df = statcast()
statcast_df



start_dt 2025-07-25
end_dt 2025-07-26
This is a large query, it may take a moment to complete


100%|██████████| 2/2 [00:03<00:00,  1.96s/it]


,pitch_type,game_date,release_speed,release_pos_x,release_pos_z,player_name,batter,pitcher,events,description,spin_dir,spin_rate_deprecated,break_angle_deprecated,break_length_deprecated,zone,des,game_type,stand,p_throws,home_team,away_team,type,hit_location,bb_type,balls,strikes,game_year,pfx_x,pfx_z,plate_x,plate_z,on_3b,on_2b,on_1b,outs_when_up,inning,inning_topbot,hc_x,hc_y,tfs_deprecated,tfs_zulu_deprecated,umpire,sv_id,vx0,vy0,vz0,ax,ay,az,sz_top,sz_bot,hit_distance_sc,launch_speed,launch_angle,effective_speed,release_spin_rate,release_extension,game_pk,fielder_2,fielder_3,fielder_4,fielder_5,fielder_6,fielder_7,fielder_8,fielder_9,release_pos_y,estimated_ba_using_speedangle,estimated_woba_using_speedangle,woba_value,woba_denom,babip_value,iso_value,launch_speed_angle,at_bat_number,pitch_number,pitch_name,home_score,away_score,bat_score,fld_score,post_away_score,post_home_score,post_bat_score,post_fld_score,if_fielding_alignment,of_fielding_alignment,spin_axis,delta_home_win_exp,delta_run_exp,bat_speed,swing_length,estimated_slg_using_speedangle,delta_pitcher_run_exp,hyper_speed,home_score_diff,bat_score_diff,home_win_exp,bat_win_exp,age_pit_legacy,age_bat_legacy,age_pit,age_bat,n_thruorder_pitcher,n_priorpa_thisgame_player_at_bat,pitcher_days_since_prev_game,batter_days_since_prev_game,pitcher_days_until_next_game,batter_days_until_next_game,api_break_z_with_gravity,api_break_x_arm,api_break_x_batter_in,arm_angle,attack_angle,attack_direction,swing_path_tilt,intercept_ball_minus_batter_pos_x_inches,intercept_ball_minus_batter_pos_y_inches
2842,FF,2025-07-25,100.0,-2.24,6.04,"Halvorsen, Seth",702616,678020,field_out,hit_into_play,<NA>,<NA>,<NA>,<NA>,2,Jackson Holliday flies out to left fielder Jor...,R,L,R,BAL,COL,X,7,fly_ball,2,2,2025,-1.04,1.16,-0.22,2.95,<NA>,<NA>,<NA>,2,9,Bot,28.6,91.34,<NA>,<NA>,<NA>,<NA>,8.104533,-145.277869,-5.909473,-16.665086,33.898785,-14.591134,3.39,1.6,357,96.1,32,100.9,2187,6.9,777020,696100,669911,642731,606115,678662,687597,666160,671289,53.62,0.209,0.361,0.0,1,0,0,5,74,5,4-Seam Fastball,5,6,5,6,6,5,5,6,Standard,Standard,220,-0.044,-0.219,71.4,6.2,0.674,0.219,96.1,-1,-1,0.044,0.044,25,21,25,22,1,4,<NA>,<NA>,<NA>,<NA>,1.13,1.04,-1.04,<NA>,3.887941,16.383637,25.786774,39.142809,21.003173
3029,FF,2025-07-25,100.6,-2.16,6.04,"Halvorsen, Seth",702616,678020,NaN,ball,<NA>,<NA>,<NA>,<NA>,13,Jackson Holliday flies out to left fielder Jor...,R,L,R,BAL,COL,B,<NA>,NaN,1,2,2025,-1.16,1.1,-1.24,2.1,<NA>,<NA>,<NA>,2,9,Bot,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5.3575,-146.286904,-8.287605,-17.863189,34.497347,-14.662045,3.4,1.58,<NA>,<NA>,<NA>,101.8,2218,7.0,777020,696100,669911,642731,606115,678662,687597,666160,671289,53.52,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,74,4,4-Seam Fastball,5,6,5,6,6,5,5,6,Standard,Standard,220,0.0,0.039,<NA>,<NA>,<NA>,-0.039,<NA>,-1,-1,0.044,0.044,25,21,25,22,1,4,<NA>,<NA>,<NA>,<NA>,1.16,1.16,-1.16,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3145,FF,2025-07-25,101.1,-2.17,6.11,"Halvorsen, Seth",702616,678020,NaN,ball,<NA>,<NA>,<NA>,<NA>,11,Jackson Holliday flies out to left fielder Jor...,R,L,R,BAL,COL,B,<NA>,NaN,0,2,2025,-1.15,1.14,-1.14,3.97,<NA>,<NA>,<NA>,2,9,Bot,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,5.655751,-147.104855,-3.37315,-18.05816,35.702867,-15.071266,3.39,1.55,<NA>,<NA>,<NA>,102.0,2254,6.8,777020,696100,669911,642731,606115,678662,687597,666160,671289,53.68,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,74,3,4-Seam Fastball,5,6,5,6,6,5,5,6,Standard,Standard,222,0.0,0.023,<NA>,<NA>,<NA>,-0.023,<NA>,-1,-1,0.044,0.044,25,21,25,22,1,4,<NA>,<NA>,<NA>,<NA>,1.1,1.15,-1.15,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
3266,FF,2025-07-25,100.6,-2.05,6.04,"Halvorsen, Seth",702616,678020,NaN,foul,<NA>,<NA>,<NA>,<NA>,3,Jackson Holliday flies out to left fielder Jor...,R,L,R,BAL,COL,S,<NA>,NaN,0,1,2025,-1.07,0.89,0.28,2.93,<NA>,<NA>,<NA>,2,9,Bot,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,9.064859,-146.197886,-5.431255,-17.484724,33.816323,-18.358337,3.39,1.6,266,82.6,31,101.5,2014,6.8,777020,696100,669911,642731,606115,678662,687597,666160,671289,53